# Database Support

## APIs and usage

## TODO

## Helper defining 

In [1]:
from IMP import *
from peewee import *
from IPython.core.debugger import Tracer; set_trace = Tracer(colors='linux')

Docstring = '''
Database relative functional module
'''

config_db = SqliteDatabase('base.db')

In [2]:
config_db.

True